In [1]:
# import the necessary libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import sklearn
import cv2
import sys
import os
import random
import tensorflow as tf
from tensorflow import keras
import pydot

In [2]:
# define the image size and image path
IMG_WIDTH=128
IMG_HEIGHT=128
img_folder=r'dataset\pictures\train\128x128'

In [ ]:
######################################################
# auxiliary code to visualize all pictures of dataset
######################################################
plt.figure(figsize=(20,20))
list_of_files=os.listdir(img_folder)
number_of_files=len(list_of_files)

for i in range(number_of_files):
    file = random.choice(os.listdir(img_folder))
    image_path= os.path.join(img_folder, file)
    img=mpimg.imread(image_path)
    ax=plt.subplot(12,12,i+1)
    ax.title.set_text(file)
    plt.imshow(img)
######################################################

In [4]:
# function to gather pictures and generate a numpy array
def create_dataset(img_folder):
   
    img_data_array = []
    class_name = []
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path = os.path.join(img_folder, dir1, file)
            image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array, class_name

In [5]:
# extract the image array and class name
X_train, y_train = create_dataset(img_folder)

In [6]:
# enumerate the classes found
target_dict={k: v for v, k in enumerate(np.unique(y_train))}
target_dict

{'cpu': 0, 'gpu': 1, 'mobo': 2, 'ram': 3}

In [7]:
###################################################################
# auxiliary code to visualize the array generated from the pictures
###################################################################
#np.set_printoptions(threshold=sys.maxsize)
print(X_train)
###################################################################

[array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       ...,

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32), array([[[0.6666667 , 0.69411767, 0.68235296],
        [0.36078432, 0.39215687, 0.2470588

In [8]:
print(y_train)

['cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'cpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'gpu', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'mobo', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram', 'ram']


In [ ]:
target_val=  [target_dict[y_train[i]] for i in range(len(y_train))]

In [ ]:
# build the neural network layers
model=tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(IMG_HEIGHT,IMG_WIDTH, 3)),
            tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(6)
        ])
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x=np.array(X_train, np.float32), y=np.array(list(map(int,target_val)), np.float32), epochs=5)

O primeiro passo é carregar um conjunto de dados. No caso, usamos aqui o _dataset_ **Fashion MNIST**, com imagens reduzidas de roupas. A biblioteca Keras tem várias funções para carregar conjuntos de dados populares em `keras.datasets`. 

O conjunto de dados já está dividido entre instâncias de treinamento e de teste, mas a seguir iremos dividir o conjunto de treinamento para ter um conjunto de validação.

Cada instância é uma imagem em tons de cinza (com valores de 0 a 255) e com resolução 28 por 28 _pixels_. Note que esse _dataset_ foi feito para ser compatível com o conjunto **MNIST** original, tendo a mesma resolução, número de instâncias e número de classes (10), porém sendo mais desafiador de classificar.

In [ ]:
# importação do dataset
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print('treinamento completo:', X_train_full.shape)
print('testes:              ', X_test.shape)

Aqui o conjunto completo de treinamento é quebrado em dois, um de treinamento menor e outro de validação. Também é feita a conversão dos valores inteiros de tons de cinza (de 0 a 255) para um vaor real no intervalo de 0 a 1.

In [ ]:
# separação dos dados de treinamento e validação
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.
print('treinamento:', X_train.shape)
print('validação:   ', X_valid.shape)

Abaixo é exibida primeira instância do conjunto de treino:

In [ ]:
plt.imshow(X_train[0], cmap='binary')
plt.show()

Os rótulos são valores inteiros de 0 a 9, quardados nos vetores `y` e que correspondem aos seguintes nomes de classes.

Então a instância 0 é um **casaco**.

In [ ]:
class_names = ['camiseta', 'calça', 'pulôver', 'vestido', 'casaco',
               'sandália', 'camisa', 'tênis', 'bolsa', 'bota']
class_names[y_train[0]]

Abaixo é exibido um mosaico com várias instâncias do conjunto de treino:

In [ ]:
# não se preocupe com este código

n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap='binary')
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

# Criando uma rede neural

Aqui vamos criar uma rede neural de classificação, usando um modelo (ou arquitetura) do tipo sequencial. O modelo sequencial corresponde ao tipo mais simples de rede neural, onde uma sequência de camadas de neurônios é empilhada uma em cima da outra.

- A criação começa com a chamada a `Sequential`, que define o tipo do modelo:

        model = keras.models.Sequential()

- Então uma camada do tipo `Flatten` é adicionada. Seu papel é apenas transformar a matriz de _pixels_ em um longo vetor. É uma camada de preprocessamento dos dados. Mas como esta é a primeira camada, é preciso definir o formato da entrada com `input_shape`:

        model.add(keras.layers.Flatten(input_shape=[28, 28]))

- A seguir adicionamos uma camada densa do tipo `Dense`, ou seja, totalmente conectada com a camada anterior. Esta conta com 300 neurônios e função de ativação ReLU:

        model.add(keras.layers.Dense(300, activation='relu'))
        
- Então uma segunda camada `Dense` é adicionada, agora com 100 neurônios e função de ativação também ReLU:
        
        model.add(keras.layers.Dense(100, activation='relu'))
        
- Finalmente uma camada de saída é adicionada. Aqui o tipo também é `Dense`, mas a função de ativação é trocada para `softmax` para produzir a saída de classificador (uma vez que as classes são mutuamente exclusivas):
        
        model.add(keras.layers.Dense(10, activation='softmax'))

Ainda que se possa criar uma rede neural com as diversas chamadas a `model.add(...)`, é mais conveniente criar o modelo passando uma lista de camadas, como mostrado a seguir.

In [ ]:
# comando para 'zerar' a biblioteca Keras
keras.backend.clear_session()

# definição de sementes aleatórias
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# especificação do modelo
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
# resumo legível da arquitetura deste modelo
model.summary()

Note que as camadas densas geralmente têm muitos parâmetros. Por exemplo, a primeira camada densa tem pesos de conexão de 784 × 300, além de mais 300 termos de _bias_, chegando a um total de 235.500 parâmetros.

Isso dá ao modelo bastante flexibilidade para ajustar os dados de treinamento, mas também significa que o modelo corre o risco de ter _overfitting_, especialmente quando não há muitos dados de treinamento.

## Arquitetura da rede neural

Podemos gerar uma figura da arquitetura deste modelo usando a função `keras.utils.plot_model`.

In [ ]:
keras.utils.plot_model(model, 'fashion_mnist_model.png', show_shapes=True)

## Acesso às camadas

A biblioteca permite acessar cada camada criada, usando índices de acesso tal como em uma lista de Python.

Permite também ver atributos de cada camada, como o nome ou se é uma camada oculta. E ainda permite inspecionar os pesos de todas as conexões daquela camada.

In [ ]:
# acesso a cada uma das camadas
model.layers

In [ ]:
# primeira camada e respectivo nome
hidden1 = model.layers[1]
hidden1.name

In [ ]:
# encontra camada pelo nome
model.get_layer(hidden1.name)

In [ ]:
# indica se a camada é ou não oculta
model.get_layer(hidden1.name) is hidden1

Observe que a camada `Dense` inicializa os pesos das conexão aleatoriamente. Os vieses foram inicializados apenas com zero.

In [ ]:
# obtém pesos e vieses da camada
weights, biases = hidden1.get_weights()
print('weights:', weights.shape)
print('biases: ', biases.shape)

In [ ]:
weights

In [ ]:
biases

## Compilando a rede neural

Depois que um modelo é criado, é preciso chamar o método `compile()`, especificando a **função de perda** (aqui, a função `sparse_categorical_crossentropy`) e o **otimizador** a ser usado (`sgd`, algoritmo de descida do gradiente estocástico).

Opcionalmente, você também pode especificar uma lista de **medidas de desempenho** extras para calcular durante o treinamento e avaliação. Neste caso apenas é indicada a acurácia com `accuracy`.

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', 
              metrics=['accuracy'])

# Treinando e avaliando a rede neural

Para treinar o modelo basta chamar o método `fit()`. 

Três parâmetros são obrigatórios: as _features_ de treinamento, os rótulos de treinamento e o número de épocas.

Cada **época** (_epoch_) corresponde a uma etapa de atualização da rede neural.

Opcionalemente é passado também um conjunto de validação. A biblioteca Keras medirá a perda e as métricas extras ao final de cada época, o que é muito útil para ver como o modelo realmente funciona: se o desempenho no conjunto de treinamento é muito melhor do que no conjunto de validação, provavelmente está ocorrendo _overfitting_.

In [ ]:
# esta chamada pode demorar um pouco
%time history = model.fit(X_train, y_train, epochs=25, validation_data=(X_valid, y_valid))

In [ ]:
# os dados do treinamento estão disponíveis no histórico retornado
print('parâmetros:', history.params)
print('métricas:  ', list(history.history.keys()))
print('épocas:    ', history.epoch)

## Visualização da evolução das métricas ao longo do treinamento

In [ ]:
pd.DataFrame(history.history).plot(figsize=(12, 6))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

## Avaliação final do modelo e geração de previsões

In [ ]:
# avaliação com conjunto de teste
model.evaluate(X_test, y_test)

In [ ]:
# probabilidades computadas para três instâncias de teste
X_new = X_test[:3]
y_proba = model.predict(X_new)
print(y_proba.round(2))

In [ ]:
# classes previstas e reais para as mesmas três instâncias de teste
y_pred = np.argmax(model.predict(X_new), axis=-1)
print('previstas: ', np.array(class_names)[y_pred])
print('reais:     ', np.array(class_names)[y_test[:3]])

In [ ]:
# visualização das tres instâncias
plt.figure(figsize=(7.2, 2.4))
for index, image in enumerate(X_new):
    plt.subplot(1, 3, index + 1)
    plt.imshow(image, cmap="binary", interpolation="nearest")
    plt.axis('off')
    plt.title(class_names[y_test[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()